# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [ ]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
forest_fires_data = pd.read_csv('../../05_src/data/fires/forestfires.csv', header=None, names=columns, skiprows=1)
forest_fires_data


# Get X and Y

Create the features data frame and target data.

In [ ]:
X_features = forest_fires_data.drop('area', axis=1)
Y_target = forest_fires_data['area']

from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y_target, test_size=0.3, random_state=42)



# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [ ]:
# preproc1
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Identify numerical and categorical columns
numerical_columns = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_columns = ['month', 'day']

In [ ]:
# preproc1

   # Numerical transformations
numerical_transformations = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Categorical transformations
categorical_transformations = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='error', drop=None))
])

# Create preprocessing pipeline #1
preprocessor1 = ColumnTransformer(
    transformers=[
        ('num_transform', numerical_transformations, numerical_columns), 
        ('cat_transform', categorical_transformations, categorical_columns)
    ]
)

preprocessor1

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [ ]:
# preproc2

from sklearn.preprocessing import FunctionTransformer, PowerTransformer
import numpy as np

# Binarize rain column
def binarize_rain(rain):
    return np.where(rain > 0, 1, 0).reshape(-1, 1)

# Rain transformation
rain_transformation = Pipeline(steps=[
    ('binarizer', FunctionTransformer(binarize_rain, validate=True))
])

# Numerical transformations
numeric_transformations_2 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('transform', PowerTransformer())
])

# Create preprocessing pipeline #2
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformations_2, numerical_columns),
        ('rain', rain_transformation, ['rain']),
        ('cat', categorical_transformations, categorical_columns)
    ]
)

preprocessor2


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Define the pipeline for preprocessing and regression
pipeline_simple = Pipeline(
    steps=[
        ('preprocess', preprocessor1),
        ('regressor', LinearRegression())
    ]
)

# Split the dataset into training and testing sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_features, Y_target, test_size=0.3, random_state=42)

# Fit the pipeline to the training data
pipeline_simple.fit(X_train_split, y_train_split)

# Define the scoring metrics
scoring_metrics = ['r2', 'neg_root_mean_squared_error']

# Perform cross-validation
cross_val_results = cross_validate(pipeline_simple, X_train_split, y_train_split, cv=5, scoring=scoring_metrics, return_train_score=True)

# Convert cross-validation results to a DataFrame
results_df_simple = pd.DataFrame(cross_val_results).assign(experiment=1)
results_df_simple


In [ ]:
# Import necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Define the pipeline for preprocessing and regression
pipeline_transform = Pipeline(
    steps=[
        ('preprocess', preprocessor2),
        ('regressor', LinearRegression())
    ]
)

# Split the dataset into training and testing sets
X_train_trans, X_test_trans, y_train_trans, y_test_trans = train_test_split(X_features, Y_target, test_size=0.3, random_state=42)

# Fit the pipeline to the training data
pipeline_transform.fit(X_train_trans, y_train_trans)

# Define the scoring metrics
scoring_metrics_trans = ['r2', 'neg_root_mean_squared_error']

# Perform cross-validation
cross_val_results_trans = cross_validate(pipeline_transform, X_train_trans, y_train_trans, cv=5, scoring=scoring_metrics_trans, return_train_score=True)

# Convert cross-validation results to a DataFrame
results_df_trans = pd.DataFrame(cross_val_results_trans).assign(experiment=2)
results_df_trans


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Define the pipeline for preprocessing and regression
pipeline_advanced1 = Pipeline(
    steps=[
        ('preprocess', preprocessor1),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ]
)

# Split the dataset into training and testing sets
X_train_adv1, X_test_adv1, y_train_adv1, y_test_adv1 = train_test_split(X_features, Y_target, test_size=0.3, random_state=42)

# Fit the pipeline to the training data
pipeline_advanced1.fit(X_train_adv1, y_train_adv1)

# Define the scoring metrics
scoring_metrics_adv1 = ['r2', 'neg_root_mean_squared_error']

# Perform cross-validation
cross_val_results_adv1 = cross_validate(pipeline_advanced1, X_train_adv1, y_train_adv1, cv=5, scoring=scoring_metrics_adv1, return_train_score=True)

# Convert cross-validation results to a DataFrame
results_df_adv1 = pd.DataFrame(cross_val_results_adv1).assign(experiment=3)
results_df_adv1


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Define the pipeline for preprocessing and regression
pipeline_advanced2 = Pipeline(
    steps=[
        ('preprocess', preprocessor2),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ]
)

# Split the dataset into training and testing sets
X_train_adv2, X_test_adv2, y_train_adv2, y_test_adv2 = train_test_split(X_features, Y_target, test_size=0.3, random_state=42)

# Fit the pipeline to the training data
pipeline_advanced2.fit(X_train_adv2, y_train_adv2)

# Define the scoring metrics
scoring_metrics_adv2 = ['r2', 'neg_root_mean_squared_error']

# Perform cross-validation
cross_val_results_adv2 = cross_validate(pipeline_advanced2, X_train_adv2, y_train_adv2, cv=5, scoring=scoring_metrics_adv2, return_train_score=True)

# Convert cross-validation results to a DataFrame
results_df_adv2 = pd.DataFrame(cross_val_results_adv2).assign(experiment=4)
results_df_adv2


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the GridSearchCV
param_grid_simple = {
    'regressor__fit_intercept': [True, False],
    'regressor__positive': [True, False]
}

# Define the scoring metrics
scoring_metrics_simple = ['r2', 'neg_root_mean_squared_error']

# Initialize GridSearchCV
grid_search_simple = GridSearchCV(
    estimator=pipeline_simple,
    param_grid=param_grid_simple,
    scoring=scoring_metrics_simple,
    cv=5,
    refit='neg_root_mean_squared_error'
)

# Fit GridSearchCV to the training data
grid_search_simple.fit(X_train_split, y_train_split)

# Print the best parameters and best score
print(grid_search_simple.best_params_)
print(grid_search_simple.best_score_)


In [ ]:
# Pipeline B

from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the GridSearchCV
param_grid_transform = {
    'regressor__fit_intercept': [True, False],
    'regressor__positive': [True, False]  # Example of another parameter
}

# Define the scoring metrics
scoring_metrics_transform = ['r2', 'neg_root_mean_squared_error']

# Initialize GridSearchCV
grid_search_transform = GridSearchCV(
    estimator=pipeline_transform,
    param_grid=param_grid_transform,
    scoring=scoring_metrics_transform,
    cv=5,
    refit='neg_root_mean_squared_error'
)

# Fit GridSearchCV to the training data
grid_search_transform.fit(X_train_trans, y_train_trans)

# Print the best parameters and best score
print(grid_search_transform.best_params_)
print(grid_search_transform.best_score_)


In [ ]:
# Pipeline C

from sklearn.model_selection import GridSearchCV

# Get parameters of Pipeline C
pipeline_c_params = pipeline_advanced1.get_params()

# Set GridSearch parameter grid
param_grid_advanced1 = {
    'regressor__n_estimators': [50, 100, 150, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10, 15],
    'regressor__max_features': ['sqrt', 'log2', None, 0.5]
}

# Define the scoring metrics
scoring_metrics_advanced1 = ['r2', 'neg_root_mean_squared_error']

# Initialize GridSearchCV
grid_search_advanced1 = GridSearchCV(
    estimator=pipeline_advanced1,
    param_grid=param_grid_advanced1,
    scoring=scoring_metrics_advanced1,
    cv=5,
    refit='neg_root_mean_squared_error'
)

# Fit GridSearchCV to the training data
grid_search_advanced1.fit(X_train_adv1, y_train_adv1)

# Print the best parameters and best score
print(grid_search_advanced1.best_params_)
print(grid_search_advanced1.best_score_)


In [ ]:
from sklearn.model_selection import GridSearchCV

# Set GridSearch parameter grid
param_grid_advanced2 = {
    'regressor__n_estimators': [50, 100, 150, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10, 15],
    'regressor__max_features': ['sqrt', 'log2', None, 0.5]
}

# Define the scoring metrics
scoring_metrics_advanced2 = ['r2', 'neg_root_mean_squared_error']

# Initialize GridSearchCV
grid_search_advanced2 = GridSearchCV(
    estimator=pipeline_advanced2,
    param_grid=param_grid_advanced2,
    scoring=scoring_metrics_advanced2,
    cv=5,
    refit='neg_root_mean_squared_error'
)

# Fit GridSearchCV to the training data
grid_search_advanced2.fit(X_train_adv2, y_train_adv2)

# Print the best parameters and best score
print(grid_search_advanced2.best_params_)
print(grid_search_advanced2.best_score_)


# Evaluate

+ Which model has the best performance?

The random forest model in Pipeline C.

# Export

+ Save the best performing model to a pickle file.

In [ ]:
import pickle

best_model = grid_search_advanced1.best_estimator_

filename = 'best_model_pipeline_c.pkl'
pickle.dump(best_model, open(filename, 'wb'))

print(f"Best model from Pipeline C saved to {filename}")

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
import shap

data_transform = best_model.named_steps['preprocessing'].transform(X_test)

# Create an explainer object
explainer = shap.Explainer(best_model.named_steps['regressor'], 
                           data_transform, 
                           feature_names=best_model.named_steps['preprocessing'].get_feature_names_out())

shap_values = explainer(data_transform, check_additivity=False)

# Visualize SHAP values for a specific observation (e.g., the second observation)
shap.plots.waterfall(shap_values[1])

*(Answer here.)*
Temperature and the DMC index are identified as the most influential predictors, consistently demonstrating strong explanatory power across all samples, while features such as ffmc, rh, and dc, which show minimal impact as indicated by clustered SHAP values near zero, warrant consideration for removal pending correlation analysis and iterative model evaluation to assess potential improvements in RMSE and R-squared.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.